In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
from PIL import Image
from sklearn.model_selection import train_test_split
from PIL import Image
from skimage.io import imread
import pandas as pd
import time


In [ ]:
# To extract the zip file. Can be skipped if already extracted
needs_extract = False
if needs_extract:
    local_zip = 'chest-xray-pneumonia.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('demo')
    zip_ref.close()
    base_path = "demo/chest_xray/test"
    folder = os.listdir(base_path)
    !rm -r 'demo/chest_xray/chest_xray/'
    !rm -r 'demo/chest_xray/__MACOSX'


In [3]:
train_normal_path = 'demo/chest_xray/train/NORMAL/'
folder = os.listdir(train_normal_path)
print("Normal Train Images ", len(folder))
total_images = len(folder)
df_normal_train =  pd.DataFrame(index=np.arange(0, total_images), columns=["path", "target"])
for i in range(total_images):
    df_normal_train.iloc[i]['path'] =train_normal_path + folder[i]
    df_normal_train.iloc[i]['target'] = 0
train_pne_path = 'demo/chest_xray/train/PNEUMONIA/'
folder = os.listdir(train_pne_path)
print("Pneumonia Train Images ", len(folder))

total_images =  len(folder)
df_pne_train =  pd.DataFrame(index=np.arange(0, total_images), columns=["path", "target"])
for i in range(total_images):
    df_pne_train.iloc[i]['path'] = train_pne_path+ folder[i]
    df_pne_train.iloc[i]['target'] = 1

RAND_STATE = 4545


Normal Train Images  1341
Pneumonia Train Images  3875


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df_train = pd.concat([df_normal_train.copy(),df_pne_train.copy()])
df_train = df_train.sample(frac = 1,random_state = RAND_STATE)
test_normal_path = 'demo/chest_xray/test/NORMAL/'
folder = os.listdir(test_normal_path)
print("Normal Test Images ", len(folder))
total_images = len(folder)
df_normal_test =  pd.DataFrame(index=np.arange(0, total_images), columns=["path", "target"])
for i in range(total_images):
    df_normal_test.iloc[i]['path'] =test_normal_path + folder[i]
    df_normal_test.iloc[i]['target'] = 0
test_pne_path = 'demo/chest_xray/test/PNEUMONIA/'
folder = os.listdir(test_pne_path)
print("Pneumonia Test Images ", len(folder))
total_images = len(folder)
df_pne_test =  pd.DataFrame(index=np.arange(0, total_images), columns=["path", "target"])
for i in range(total_images):
    df_pne_test.iloc[i]['path'] =test_pne_path + folder[i]
    df_pne_test.iloc[i]['target'] = 1

Normal Test Images  234
Pneumonia Test Images  390


In [5]:
df_test = pd.concat([df_normal_test.copy(),df_pne_test.copy()])
df_test = df_test.sample(frac = 1,random_state = RAND_STATE)

In [6]:
def my_transform(key="train"):
    train_sequence = [transforms.Resize((224,224)),                      
                      transforms.ToTensor()]
    test_sequence = [transforms.Resize((224,224)),
                    transforms.ToTensor()]
    data_transforms = {'train': transforms.Compose(train_sequence),
                       'test': transforms.Compose(test_sequence)}
    return data_transforms[key]


class PNEDataset(Dataset):
    
    def __init__(self, df, transform=None):
        self.states = df
        self.transform=transform
      
    def __len__(self):
        return len(self.states)
        
    def __getitem__(self, idx):
        image_path = self.states.path.values[idx]
        image = Image.open(image_path)
        image = image.convert('L')
        
        if self.transform:
            image = self.transform(image)
         
        target = np.int(self.states.target.values[idx])
        return image,target

train_dataset = PNEDataset(df_train, transform=my_transform(key="train"))
test_dataset =  PNEDataset(df_test, transform=my_transform(key="test"))

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, drop_last=False)

In [7]:
torch.manual_seed(756)
model = torchvision.models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256,9)
)
model = model.to(device)

In [8]:
def cumLaplaceDistribution(y_pred,mean,standard_deviation,all_qs):
    term1 = ((1-all_qs) * (y_pred - mean))/standard_deviation
    term1.clamp_(max = 0)
    lesser_term = all_qs * torch.exp(term1)
    term2 = (-1.0 * all_qs * (y_pred - mean))/standard_deviation
    term2.clamp_(max = 0)
    greater_term = 1 - ((1-all_qs) * torch.exp(term2))
    dummy_ones = torch.ones_like(mean)
    y_dummy_pred = torch.div(y_pred,dummy_ones)
    y_dummy_pred[y_pred >= mean] = 1.0
    y_dummy_pred[y_pred < mean] = 0.0
    return ((1 - y_dummy_pred) * lesser_term )+  (y_dummy_pred * greater_term)


def logLikelihoodLoss(y_true,y_pred,mean,standard_deviation,all_qs):
    new_pred = y_pred
    prob_tens = cumLaplaceDistribution(0.0,mean = new_pred,standard_deviation = standard_deviation,all_qs = all_qs)
    prob_tens.clamp_(min = 1e-7,max = 1 - 1e-7)
    if_one = y_true * torch.log(1 - prob_tens)
    if_zero = (1 - y_true) * torch.log(prob_tens)
    result = - 1 * torch.mean(if_one + if_zero)
    return result

def customLoss(y_true, y_pred, mean, standard_deviation, all_qs, penalty):
    ind_losses = []
    for i,j in enumerate(all_qs):
        solo_loss = logLikelihoodLoss(y_true[:,0],y_pred[:,i] ,mean, standard_deviation, j)
        ind_losses.append(solo_loss)
    zero = torch.Tensor([0]).to(device)
    dummy1 = y_pred[:,1:] - y_pred[:,:-1]
    dummy2 = penalty * torch.mean(torch.max(zero,-1.0 * dummy1))
    total_loss  = torch.mean(torch.stack(ind_losses)) +dummy2
    return total_loss

def customTestPred(y_pred,mean,standard_deviation,all_qs,batch_size = 1):
    if(batch_size == 1):
        acc = []
        cdfs = []
        eps = 1e-10
        val = (y_pred - mean)/standard_deviation 
        for xx in range(batch_size):
            if(y_pred < mean.item()):
                lesser_term = all_qs * torch.exp((1 - all_qs) * val.item())
                lesser_term  = 1 - lesser_term
                cdfs.append(lesser_term.item())
                if(lesser_term.item() >= 0.5):
                    acc.append([1])
                else:
                    acc.append([0])
            
            elif(y_pred >= mean.item()):
                greater_term = 1 - ((1-all_qs) * torch.exp(-1 * all_qs * val.item()))
                greater_term = 1 - greater_term
                cdfs.append(greater_term.item())
                if(greater_term.item() >= 0.5):
                    acc.append([1])
                else:
                    acc.append([0])
    
    elif(batch_size > 1):
        acc = []
        cdfs = []
        eps = 1e-10
        val = (y_pred - mean)/standard_deviation 
        for xx in range(batch_size):
            if(y_pred < mean[xx]):
                lesser_term = all_qs * torch.exp((1 - all_qs) * val[xx])
                lesser_term  = 1 - lesser_term
                cdfs.append(lesser_term.item())
                if(lesser_term.item() >= 0.5):
                    acc.append([1])
                else:
                    acc.append([0])
            elif(y_pred >= mean[xx]):
                greater_term = 1 - ((1-all_qs) * torch.exp(-1 * all_qs * val[xx]))
                greater_term = 1 - greater_term
                cdfs.append(greater_term.item())
                if(greater_term.item() >= 0.5):
                    acc.append([1])
                else:
                    acc.append([0])
    return torch.Tensor(acc).to(device).reshape(-1,1),torch.Tensor(cdfs).to(device).reshape(-1,1)

def acc_Q(train_preds,train_labels):
    train_preds = np.array(train_preds).reshape(-1,1)
    train_labels = np.array(train_labels).reshape(-1,1)

    cdfs_acc,_ = customTestPred(0,train_preds,standard_deviation = 1,all_qs = torch.Tensor([0.5]),
                                batch_size = train_preds.shape[0])

    count = 0
    for i,j in zip(cdfs_acc,train_labels):
        if(i.item() == j[0]):
            count += 1
    return count/train_labels.shape[0]

def acc_tests(test_preds,test_labels):
    test_preds = np.array(test_preds).reshape(-1,1)
    test_labels = np.array(test_labels).reshape(-1,1)
    cdfs_acc,_ = customTestPred(0,test_preds,standard_deviation = 1,all_qs = torch.Tensor([0.5]),
                                batch_size = test_preds.shape[0])

    count = 0
    for i,j in zip(cdfs_acc,test_labels):
        if(i.item() == j[0]):
            count += 1
    return count/test_labels.shape[0]


In [9]:
def train(model,optimizer,loader,epochs, verbose=True):
    train_preds_Q = []
    train_preds_bce = []
    train_labels = []
    model.train()
    for i,j in enumerate(loader):
        inputs,labels = j[0],j[1]
        inputs = inputs.to(device)
        labels = labels.to(device)
        batch_size = labels.shape
        optimizer.zero_grad()
        op_qs = model(inputs)
        lossQ = customLoss(labels.reshape(-1,1),op_qs, mean_is,std_is,all_qs,penalty)
        lossQ.backward()
        optimizer.step()
        for lag in op_qs[:,4].detach().reshape(-1,1):
            train_preds_Q.append(lag.item())
        for lag in labels.reshape(-1,1):
            train_labels.append(lag.item())
    
    acc_is_Q = acc_Q(train_preds_Q,train_labels)
    if verbose:
        print("[%d/%d] Train Acc Q : %f "%(epochs,total_epochs,acc_is_Q))
    return acc_is_Q

In [10]:
def test(model,loader,epochs,verbose=True):
    model.eval()
    test_preds_Q = []
    test_preds_bce = []
    test_labels = []
    with torch.no_grad():
        for i,j in enumerate(loader):
            inputs,labels = j[0],j[1]
            inputs = inputs.to(device)
            labels = labels.to(device)
            op_qs = model(inputs)
            for lag in op_qs[:,4].detach().reshape(-1,1):
                test_preds_Q.append(lag.item())
            for lag in labels.reshape(-1,1):
                test_labels.append(lag.item())
                
    acc_is_Q = acc_tests(test_preds_Q,test_labels)
    print("[%d/%d] Test Acc Q : %f  "%(epochs,total_epochs,acc_is_Q))




In [11]:
def quantileCDF(x, tau):
    if x>0:
        return 1 - tau*np.exp((tau-1)*x)
    else:
        return (1 - tau)*np.exp(tau*x)

In [12]:
lr_is = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr = lr_is)
all_qs = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
all_qs = torch.Tensor(all_qs).to(device)
mean_is = 0
std_is = 1
penalty = 1
epsilon = 0.00
total_epochs = 10
verbosity = True

In [13]:
for i in range(total_epochs):
    acc_train = train(model,optimizer, train_dataloader,i+1,verbosity)
    acc_test = test(model,test_dataloader,i+1,verbosity)

[1/10] Train Acc Q : 0.794095 
[1/10] Test Acc Q : 0.625000  
[2/10] Train Acc Q : 0.855253 
[2/10] Test Acc Q : 0.759615  
[3/10] Train Acc Q : 0.876534 
[3/10] Test Acc Q : 0.809295  
[4/10] Train Acc Q : 0.891104 
[4/10] Test Acc Q : 0.823718  
[5/10] Train Acc Q : 0.900690 
[5/10] Test Acc Q : 0.750000  
[6/10] Train Acc Q : 0.908167 
[6/10] Test Acc Q : 0.831731  
[7/10] Train Acc Q : 0.918137 
[7/10] Test Acc Q : 0.668269  
[8/10] Train Acc Q : 0.929256 
[8/10] Test Acc Q : 0.778846  
[9/10] Train Acc Q : 0.937500 
[9/10] Test Acc Q : 0.695513  
[10/10] Train Acc Q : 0.944594 
[10/10] Test Acc Q : 0.793269  


In [14]:
delta_total = [0,0,0,0,0]
delta_misc = [0,0,0,0,0]
accept_total = [0,0,0,0,0]
with torch.no_grad():
    all_preds = [[] for i in range(9)]
    test_labels = []
    for i,j in test_dataloader:
        inputs,labels = i.to(device),j.to(device)
        op_qs = model(inputs)
        for itemset in op_qs.detach():
            for quant in range(9):
                all_preds[quant].append(itemset[quant].item())
        for lbl in labels.reshape(-1,1):
            test_labels.append(lbl.item())
    
    for i,j in train_dataloader:
        inputs,labels = i.to(device),j.to(device)
        op_qs = model(inputs)
        for itemset in op_qs.detach():
            for quant in range(9):
                all_preds[quant].append(itemset[quant].item())
        for lbl in labels.reshape(-1,1):
            test_labels.append(lbl.item())

correct_counter = 0
for i in range(len(test_labels)):
    start = 4
    left = start
    right = start
    found = False
    count = 0
    medprob = quantileCDF(all_preds[start][i], 0.5)
    while (left>-1 and not found):
        q_left = all_preds[left][i]
        q_right = all_preds[right][i]
        p_left = quantileCDF(q_left, 0.5)
        p_right = quantileCDF(q_right, 0.5)
        left -=1
        right +=1
        if (q_left < 0.5 and q_right>0.5):
            found = True
        else:
            count +=1
    delta_total[count-1] +=1
    for temp in range(5):
        if count-1>=temp:
            accept_total[temp] +=1
    if (test_labels[i]==0 and medprob<=0.5) or (test_labels[i]==1 and medprob>0.5):
        correct_pred = True
        correct_counter += 1
    else:
        correct_pred = False
    if not correct_pred:
        delta_misc[count-1] +=1


In [15]:
print("Accuracy:{:.2f}".format(correct_counter/len(test_labels)))
print()

print("Delta      |",end=" ")
for i in range(5):
    print("{:.2f}".format(0.1*(i+1)), end=" | ")
print()
print("Misc. Rate |",end=" ")
for i in range(5):
    if delta_total[i] != 0:
        val = delta_misc[i]/delta_total[i]
    else:
        val = 0
    print("{:.2f}".format(val), end=" | ")
print()
print("Ret.  Rate |",end=" ")
for i in accept_total:
    print("{:.2f}".format(i/len(test_labels)), end=" | ")

Accuracy:0.94

Delta      | 0.10 | 0.20 | 0.30 | 0.40 | 0.50 | 
Misc. Rate | 0.30 | 0.26 | 0.13 | 0.03 | 0.01 | 
Ret.  Rate | 1.00 | 0.94 | 0.86 | 0.76 | 0.46 | 